In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df=pd.read_csv("covid_data.csv")

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum() #missing values

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
df['gender'].value_counts() # nominal categorical value

Female    59
Male      41
Name: gender, dtype: int64

In [7]:
df['city'].value_counts()# nominal categorical value

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [8]:
df['cough'].value_counts() #ordinal categorical value

Mild      62
Strong    38
Name: cough, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [10]:
X_train.head()

,age,gender,fever,cough,city
62,56,Female,104.0,Strong,Bangalore
57,49,Female,99.0,Strong,Bangalore
7,20,Female,NaN,Strong,Mumbai
96,51,Female,101.0,Strong,Kolkata
86,25,Male,104.0,Mild,Bangalore


In [11]:
y_train.head()

62    Yes
57     No
7     Yes
96    Yes
86    Yes
Name: has_covid, dtype: object

# Without Column Transformer

In [12]:
#Simpleimputer to fever column
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.transform(X_test[['fever']])

In [13]:
#ordinal encoding to cough column
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [14]:
#one hot encoding to column. During one hot encoding, take care of multicollinearity
oe=OneHotEncoder(drop='first',sparse=False)
X_train_gender_city=oe.fit_transform(X_train[['gender','city']])
X_test_gender_city=oe.transform(X_test[['gender','city']])
X_test_gender_city.shape

(20, 4)

In [15]:
#Extracting Age
X_train_age=X_train.drop(columns=['gender','fever','cough','city'])
X_test_age=X_test.drop(columns=['gender','fever','cough','city'])

In [16]:
#Concatenating
X_train_transformed=np.concatenate((X_train_age,X_train_cough,X_train_fever,X_train_gender_city),axis=1)
X_test_transformed=np.concatenate((X_test_age,X_test_cough,X_test_fever,X_test_gender_city),axis=1)
X_train_transformed.shape


(80, 7)

# With Column Transformer

In [17]:
from sklearn.compose import ColumnTransformer

In [18]:
#list of tuples are passed
ct=ColumnTransformer(transformers=[
    ('tf1',SimpleImputer(),['fever']), #tf1=ttuple name, transformerclass, column name
    ('tf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tf3',OneHotEncoder(drop='first', sparse=False),['gender','city'])],remainder='passthrough')
#remainder='passthrough' or 'drop', as we are not doing any transformation in age column, it will keep it as it is.If used drop, it will drop the column

In [22]:
ct.fit_transform(X_train).shape


(80, 7)

In [23]:
ct.fit_transform(X_test).shape

(20, 7)